First - set up the project directory structure and create functions to save the results at the end of each section.

In [ ]:
import pickle
from pathlib import Path

# Create directory structure
project_dir = Path('fmri_project')
checkpoints_dir = project_dir / 'checkpoints'
results_dir = project_dir / 'results'

# Create directories if they don't exist
for dir_path in [checkpoints_dir, results_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

def save_checkpoint(data, filename):
    """Save data to checkpoint file"""
    filepath = checkpoints_dir / filename
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)
    print(f"Checkpoint saved: {filepath}")

def load_checkpoint(filename):
    """Load data from checkpoint file"""
    filepath = checkpoints_dir / filename
    if filepath.exists():
        with open(filepath, 'rb') as f:
            return pickle.load(f)
    return None

Create events DataFrame from the trial information in the original "README" file

In [ ]:
import pandas as pd

# Check if events DataFrame already exists
events = load_checkpoint('events.pkl')

if events is None:
    # Create events DataFrame with timing information
    events = pd.DataFrame({
        'onset': [0, 60, 120, 180, 240, 300, 360, 420, 480, 540],
        'duration': [30] * 10,
        'emotion': ['calm', 'afraid', 'delighted', 'depressed',
                    'excited', 'delighted', 'depressed', 'calm',
                    'excited', 'afraid']
    })

    # Map emotions to valence categories
    valence_mapping = {
        'calm': 'neutral',
        'afraid': 'negative',
        'delighted': 'positive',
        'depressed': 'negative',
        'excited': 'positive'
    }
    events['trial_type'] = events['emotion'].map(valence_mapping)

    # Save events DataFrame
    save_checkpoint(events, 'events.pkl')
else:
    print("Loaded existing events DataFrame")

Creat Region of interest(ROI) masks

In [ ]:
from nilearn import image, input_data
from nilearn.datasets import fetch_atlas_harvard_oxford

# Check if ROI masks already exist
roi_masks = load_checkpoint('roi_masks.pkl')

if roi_masks is None:
    # Fetch Harvard-Oxford atlases
    cortical = fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
    subcortical = fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')

    # Create ROI masks
    roi_masks = {
        # Subcortical regions
        'amygdala_left': image.math_img('img == 10', img=subcortical.maps),
        'amygdala_right': image.math_img('img == 20', img=subcortical.maps),
        'hippocampus_left': image.math_img('img == 9', img=subcortical.maps),
        'hippocampus_right': image.math_img('img == 19', img=subcortical.maps),

        # Cortical regions
        'acc': image.math_img('img == 29', img=cortical.maps),
        'sup_frontal_gyrus': image.math_img('img == 3', img=cortical.maps),
        'mid_frontal_gyrus': image.math_img('img == 4', img=cortical.maps),
    }

    # Create bilateral masks
    bilateral_pairs = [
        ('amygdala', 'amygdala_left', 'amygdala_right'),
        ('hippocampus', 'hippocampus_left', 'hippocampus_right'),
    ]

    # Combine bilateral pairs
    for name, left, right in bilateral_pairs:
        roi_masks[f'{name}_bilateral'] = image.math_img(
            "img1 + img2",
            img1=roi_masks[left],
            img2=roi_masks[right]
        )

    # Save ROI masks
    save_checkpoint(roi_masks, 'roi_masks.pkl')
else:
    print("Loaded existing ROI masks")

Process individual subjects and save the results

In [ ]:
from nilearn.glm.first_level import FirstLevelModel

# Check if subject results already exist
all_subject_contrasts = load_checkpoint('subject_contrasts.pkl')
all_subject_roi_signals = load_checkpoint('subject_roi_signals.pkl')

if all_subject_contrasts is None or all_subject_roi_signals is None:
    # Initialize lists to store results
    n_subjects = 40
    all_subject_contrasts = []
    all_subject_roi_signals = []

    # Process each subject
    for subject_id in range(1, n_subjects + 1):
        subject_id_str = f'{subject_id:02d}'
        print(f'Processing subject {subject_id_str}...')

        # Check if individual subject results exist
        subject_results = load_checkpoint(f'subject_{subject_id_str}_results.pkl')

        if subject_results is None:
            # Load subject data
            func_file = f'ds005700/sub-{subject_id_str}/func/sub-{subject_id_str}_task-fe_bold.nii.gz'
            anat_file = f'ds005700/sub-{subject_id_str}/anat/sub-{subject_id_str}_T1w.nii.gz'

            func_img = image.load_img(func_file)
            anat_img = image.load_img(anat_file)

            # Create and fit first-level model
            model = FirstLevelModel(
                t_r=2.02697,  # Time repetition from .json file
                noise_model='ar1',
                standardize=True,
                hrf_model='spm',
                drift_model='cosine'
            )

            model.fit(func_img, events)

            # Define and compute contrasts
            contrasts = {
                'positive_vs_neutral': 'positive - neutral',
                'negative_vs_neutral': 'negative - neutral',
                'positive_vs_negative': 'positive - negative'
            }

            contrast_maps = {}
            for contrast_id, contrast_def in contrasts.items():
                contrast_maps[contrast_id] = model.compute_contrast(contrast_def)

            # Extract ROI signals
            roi_signals = {}
            for roi_name, roi_mask in roi_masks.items():
                masker = input_data.NiftiMasker(mask_img=roi_mask)
                roi_signals[roi_name] = masker.fit_transform(func_img)

            # Save individual subject results
            subject_results = {'contrasts': contrast_maps, 'roi_signals': roi_signals}
            save_checkpoint(subject_results, f'subject_{subject_id_str}_results.pkl')
        else:
            print(f"Loaded existing results for subject {subject_id_str}")
            contrast_maps = subject_results['contrasts']
            roi_signals = subject_results['roi_signals']

        all_subject_contrasts.append(contrast_maps)
        all_subject_roi_signals.append(roi_signals)

    # Save all subject results
    save_checkpoint(all_subject_contrasts, 'subject_contrasts.pkl')
    save_checkpoint(all_subject_roi_signals, 'subject_roi_signals.pkl')
else:
    print("Loaded existing subject results")

second-level group analysis

In [ ]:
import numpy as np
import pandas as pd
from nilearn.glm.second_level import SecondLevelModel
from nilearn import plotting, image
from scipy.stats import norm

# Check if group results already exist
group_results = load_checkpoint('group_results.pkl')

if group_results is None:
    # Create design matrix for second-level analysis
    n_subjects = 40
    design_matrix = pd.DataFrame({
        'intercept': np.ones(n_subjects),
    })

    # Initialize second-level model with explicit parameters
    second_level_model = SecondLevelModel(
        smoothing_fwhm=8.0,  # Add explicit smoothing
        n_jobs=1,  # Ensure single job for stability
        memory=None,  # Don't cache computations
        verbose=0  # Reduce verbosity
    )

    # Define contrasts to analyze
    contrasts_to_analyze = [
        'positive_vs_neutral',
        'negative_vs_neutral',
        'positive_vs_negative'
    ]

    # Initialize dictionary to store results
    group_results = {}

    # Process each contrast
    for contrast_name in contrasts_to_analyze:
        print(f"Running group analysis for {contrast_name}...")

        # Extract and preprocess contrast maps for all subjects
        contrast_maps_list = []

        # Get the first map to use as reference
        first_map = all_subject_contrasts[0][contrast_name]
        reference_affine = first_map.affine
        reference_shape = first_map.shape[:3]  # Exclude time dimension if present

        # Process each subject's contrast map
        for subject_contrasts in all_subject_contrasts:
            contrast_img = subject_contrasts[contrast_name]

            # Ensure 3D
            if len(contrast_img.shape) == 4:
                contrast_img = image.index_img(contrast_img, 0)

            # Resample to match reference with explicit parameters
            resampled_img = image.resample_img(
                contrast_img,
                target_affine=reference_affine,
                target_shape=reference_shape,
                interpolation='continuous',
                force_resample=True,
                copy_header=True,
                clip=True
            )

            contrast_maps_list.append(resampled_img)

        try:
            # Fit the second-level model
            second_level_model.fit(contrast_maps_list, design_matrix=design_matrix)

            # Compute the group-level contrast
            z_map = second_level_model.compute_contrast(
                output_type='z_score',
                second_level_stat_type='t'  # Explicitly specify t-test
            )
            p_map = second_level_model.compute_contrast(
                output_type='p_value',
                second_level_stat_type='t'
            )

            # Store results
            group_results[contrast_name] = (z_map, p_map)

        except Exception as e:
            print(f"Error processing contrast {contrast_name}: {str(e)}")
            continue

    # Save group results
    save_checkpoint(group_results, 'group_results.pkl')
else:
    print("Loaded existing group results")

Plot the contrasts results

In [ ]:
import matplotlib.pyplot as plt

# Plotting parameters
z_threshold = 3.1  # Corresponds to p < 0.001
p_val = norm.sf(z_threshold)  # Convert Z-score to p-value
print(f"Z-score threshold {z_threshold} corresponds to p-value < {p_val:.3e}")

# Plot results for each contrast
for contrast_name, (z_map, p_map) in group_results.items():
    print(f"Plotting results for {contrast_name}...")

    # Create figure with multiple views
    fig = plt.figure(figsize=(15, 5))

    # Plot sagittal, coronal, and axial views with statistical threshold
    display = plotting.plot_stat_map(
        z_map,
        threshold=z_threshold,  # Apply threshold only during visualization
        display_mode='ortho',
        title=f'Group-level {contrast_name}\n'
              f'(threshold: z>{z_threshold}, p<{p_val:.3e})',
        figure=fig
    )

    # Save the figure
    plt.savefig(f'results/group_analysis_{contrast_name}(Z scores).png',
                dpi=300,
                bbox_inches='tight')
    plt.close()